<a href="https://colab.research.google.com/github/knc6/jarvis-tools-notebooks/blob/master/jarvis-tools-notebooks/JARVIS_ML_TrainingGPU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Following example shows developing high-quality JARVIS-ML model development


In [1]:
!git clone --recursive https://github.com/Microsoft/LightGBM
!pip install jarvis-tools
!cd LightGBM && rm -rf build && mkdir build && cd build && cmake -DUSE_GPU=1 ../../LightGBM && make -j4 && cd ../python-package && python3 setup.py install --precompile --gpu;


Cloning into 'LightGBM'...
remote: Enumerating objects: 25, done.
remote: Counting objects: 100% (25/25), done.
remote: Compressing objects: 100% (24/24), done.
remote: Total 18254 (delta 7), reused 5 (delta 1), pack-reused 18229
Receiving objects: 100% (18254/18254), 12.26 MiB | 13.42 MiB/s, done.
Resolving deltas: 100% (13323/13323), done.
Submodule 'include/boost/compute' (https://github.com/boostorg/compute) registered for path 'compute'
Cloning into '/content/LightGBM/compute'...
remote: Enumerating objects: 21728, done.        
remote: Total 21728 (delta 0), reused 0 (delta 0), pack-reused 21728        
Receiving objects: 100% (21728/21728), 8.51 MiB | 9.00 MiB/s, done.
Resolving deltas: 100% (17565/17565), done.
Submodule path 'compute': checked out '36c89134d4013b2e5e45bc55656a18bd6141995a'
     |████████████████████████████████| 798kB 6.3MB/s 
     |████████████████████████████████| 296kB 18.7MB/s 
-- The C compiler identification is GNU 7.5.0
-- The CXX compiler identificatio

In [2]:
!nvidia-smi

Sat Jun 27 22:08:15 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.36.06    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   43C    P0    28W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [17]:
from jarvis.ai.pkgs.lgbm.regression import parameters_dict
from scipy.stats import median_absolute_deviation as mad
from jarvis.ai.pkgs.utils import get_ml_data
import lightgbm as lgb
import numpy as np
from sklearn.model_selection import train_test_split
from jarvis.ai.pkgs.utils import regr_scores

params = parameters_dict()

print (params)

{'optb88vdw_bandgap': {'n_estimators': 324, 'learning_rate': 0.06414333047469417, 'num_leaves': 31}, 'mbj_bandgap': {'n_estimators': 210, 'learning_rate': 0.04727272041771037, 'num_leaves': 121}, 'epsx': {'n_estimators': 139, 'learning_rate': 0.10098329400041395, 'num_leaves': 527}, 'epsy': {'n_estimators': 161, 'learning_rate': 0.264679564828344, 'num_leaves': 29}, 'epsz': {'n_estimators': 161, 'learning_rate': 0.264679564828344, 'num_leaves': 29}, 'mepsx': {'n_estimators': 75, 'learning_rate': 0.05374708509141705, 'num_leaves': 242}, 'mepsy': {'n_estimators': 120, 'learning_rate': 0.12048289662270327, 'num_leaves': 398}, 'mepsz': {'n_estimators': 89, 'learning_rate': 0.09718152788954888, 'num_leaves': 938}, 'encut': {'n_estimators': 376, 'learning_rate': 0.08982089572506267, 'num_leaves': 762}, 'kpoint_length_unit': {'n_estimators': 236, 'learning_rate': 0.03234907667844313, 'num_leaves': 794}, 'bulk_modulus_kv': {'n_estimators': 380, 'learning_rate': 0.08621497083536021, 'num_leaves

In [21]:
%%time
mem=[]
for i,j in params.items():
    X,y,jid=get_ml_data(dataset="cfid_3d", ml_property=i)
    lgbm = lgb.LGBMRegressor(device= 'gpu',n_estimators= j['n_estimators'],learning_rate= j['learning_rate'],num_leaves= j['num_leaves'])
    if 'eps' in i:#fit refractive index, not dielectric constant
      y=np.sqrt(y)
    X_train, X_test, y_train, y_test, jid_train, jid_test = train_test_split(X, y, jid, random_state=1, test_size=.1)
    %%timeit lgbm.fit(X_train,y_train)
    pred = lgbm.predict(X_test)
    reg_sc = regr_scores(y_test, pred)
    mae=reg_sc['mae'] #mean absolute error
    madev=mad(y) #mean absolute deviation
    #mae_over_madev=float(mae)/float(madev)
    mem.append([i,len(X),mae,madev])
    print ('Property,Length, MAE,MAD', i,len(X),mae,madev)
    print ()
    print ()
    

1 loop, best of 3: 58.7 s per loop
Property,Length, MAE,MAD optb88vdw_bandgap 35932 0.295472383011638 0.0


1 loop, best of 3: 1min 38s per loop
Property,Length, MAE,MAD mbj_bandgap 9769 0.5040659951020833 2.3024778


1 loop, best of 3: 3min 20s per loop
Property,Length, MAE,MAD epsx 16180 0.6394626656392877 1.9096697011468522


1 loop, best of 3: 22.1 s per loop
Property,Length, MAE,MAD epsy 16182 0.6993063623033852 1.8683189626222647


1 loop, best of 3: 22.5 s per loop
Property,Length, MAE,MAD epsz 17497 0.7191836355778838 1.822781394464261


1 loop, best of 3: 54.1 s per loop
Property,Length, MAE,MAD mepsx 9269 0.6019011183108844 1.0088598178244454


1 loop, best of 3: 2min 5s per loop
Property,Length, MAE,MAD mepsy 9273 0.5957537507842435 0.993149021955325


1 loop, best of 3: 1min 35s per loop
Property,Length, MAE,MAD mepsz 9739 0.607303091871831 1.0239570491077246


1 loop, best of 3: 17min 49s per loop
Property,Length, MAE,MAD encut 35835 68.65102165053635 370.65


1 loop, best

In [ ]:
mem_2=[]
for i,j in params.items():
  
    X,y,jid=get_ml_data(dataset="cfid_3d", ml_property=i)
    lgbm = lgb.LGBMRegressor(device= 'gpu',n_estimators= 2*j['n_estimators'],learning_rate= j['learning_rate']/2,num_leaves= 2*j['num_leaves'])
    X_train, X_test, y_train, y_test, jid_train, jid_test = train_test_split(X, y, jid, random_state=1, test_size=.1)
    %time lgbm.fit(X_train,y_train)
    pred = lgbm.predict(X_test)
    reg_sc = regr_scores(y_test, pred)
    mae=reg_sc['mae'] #mean absolute error
    madev=mad(y) #mean absolute deviation
    #mae_over_madev=float(mae)/float(madev)
    mem_2.append([i,len(X),mae,madev])
    print ('Property,Length, MAE,MAD', i,len(X),mae,madev)
    print ()
    print ()
    